In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np
from torch.utils.data import TensorDataset, DataLoader

In [20]:
batch_size = 64
epochs = 30
latent_dim = 256
num_samples = 10000
data_path = 'data/fra-eng/fra.txt'

In [113]:
class Dataset(torch.utils.data.Dataset):
    
    def __init__(self, data_path, num_samples): #, train=True, train_split=0.9, train_days=30):
        
        
        self.input_texts = []
        self.target_texts = []

        input_vocab = set()
        output_vocab = set()
        
        with open(data_path, 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')
        for line in lines[: min(num_samples, len(lines) - 1)]:
            input_text, target_text, _ = line.split('\t')
            target_text = '\t' + target_text + '\n'
            self.input_texts.append(input_text)
            for word in input_text.split():
                input_vocab.add(word.strip())
            self.target_texts.append(target_text)
            for word in target_text.split():
                output_vocab.add(word.strip())
                
        self.input_vocab2index = {word: i+2 for i, word in enumerate(input_vocab)}
        self.output_vocab2index = {word: i+2 for i, word in enumerate(output_vocab)}

    def __len__(self):
        return len(self.input_texts)
    
    def len_vocab2index(self):
        return len(self.input_vocab2index), len(self.output_vocab2index)
    
    def __getitem__(self, idx):
        return self.tensorsFromSent(self.input_texts[idx], self.target_texts[idx])
    
    def __indexesFromSentence(self, sentence, vocab):
        return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

    def __tensorFromSentence(self, sentence, vocab):
        indexes = self.__indexesFromSentence(sentence, vocab)
        indexes.append(1)
        return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

    def tensorsFromSent(self, input_sentences, output_sentences):
        input_tensor = self.__tensorFromSentence(input_sentences, self.input_vocab2index)
        target_tensor = self.__tensorFromSentence(output_sentences, self.output_vocab2index)
        return (input_tensor, target_tensor)

In [114]:
train_dataset = Dataset(data_path, 10000)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

In [107]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [108]:
class DecoderRNN(nn.Module):
    
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, input_size)
        
        
    def forward(self, input_, hidden, encoder_outputs):
        embedded = self.embedding(input_).view(1, 1, -1)
        output, hidden = self.lstm(embedded, hidden.view(1, 1, 1, -1))
        output = F.log_softmax(self.out(output[0]), dim=1)
        return output

In [109]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [110]:
def train(train_loader, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()
    
    for data in train_loader:

        input_tensor, target_tensor = data[0], data[1]
        
        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()
        
        input_length = input_tensor.size(0)
        target_length = target_tensor.size(0)
        
        encoder_outputs = torch.zeros(max_length, encoder.hidden_size)
        
        loss = 0

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0, 0]

        decoder_input = torch.tensor([[0]])

        decoder_hidden = encoder_hidden

        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == 1:
                break

        loss.backward()

        encoder_optimizer.step()
        decoder_optimizer.step()

        return loss.item() / target_length

In [111]:
len_in_v2i, len_out_v2i = train_dataset.len_vocab2index()

encoder = EncoderRNN(len_in_v2i+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len_out_v2i+2, dropout_p=0.1)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)

criterion = nn.NLLLoss()

print_loss_total = 0
for i in range(1000):

    loss = train(train_loader, encoder,
               attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
    print_loss_total += loss
    
    print_loss_avg = print_loss_total / 1
    print_loss_total = 0
    print('(%d %d%%) %.4f' % (i, i / 10 * 100, print_loss_avg))


TypeError: list indices must be integers or slices, not tuple

In [115]:
train_dataset[1]

(tensor([[1902],
         [   1]]),
 tensor([[919],
         [266],
         [  1]]))

In [94]:
train_dataset.input_texts[1]

'Hi.'

In [95]:
train_dataset.target_texts[1]

'\tSalut !\n'